In [ ]:
import pandas as pd

1. Load data from WFM CSV file.

In [33]:
df = pd.read_csv(
    r"C:\Users\a817628\OneDrive - ATOS\Desktop\WFM Eurocontrol MNP 2025.csv",
    delimiter=';' 
)
# show the data
df.head(0).iloc[:, 5:]

,Country,FirstName,LastName,Employee Last name + First name,"Employee First name + Last name (without ""LEFT"")",Gender,StartDate,EndDate,Dummy\n/\nRoma Nr.,WBS,...,Replaceable,Note,Email address,External (BE) Contract End Date,PM,Clause Travel expense in SA,das profile,Overtime allowed,Standby allowed,active_flag


2. Drop columns to match SQL bronze layer structure.

In [12]:
df.columns

Index(['DAS ID', 'NESSIE ID', 'Internal or external employee', 'FL/Subco',
       'Subco Agency', 'Country', 'FirstName', 'LastName',
       'Employee Last name + First name',
       'Employee First name + Last name (without "LEFT")', 'Gender',
       'StartDate', 'EndDate', 'Dummy\n/\nRoma Nr.', 'WBS',
       'Jan 25 FTE Count ', 'Feb 25 FTE Count ', 'Mar 25 FTE Count ',
       'Apr 25 FTE Count ', 'May 25 FTE Count ', 'Jun 25 FTE Count',
       'Jul 25 FTE Count ', 'Aug 25 FTE Count ', 'Sep 25 FTE Count ',
       'Oct 25 FTE Count ', 'Nov 25 FTE Count ', 'Dec 25 FTE Count ',
       'Customer n°', 'Customer', 'Contract', 'Contract Type', 'GCM Level',
       'Hourly PURCHASE PRICE', 'Hourly\nSales Rate', '% Margin', 'Profile',
       'Competence', 'Division', 'Location', 'Contractual Profile (10.A)',
       'Seniority', ' Sales ', ' Cost ', 'Margin', 'Total cost/month',
       'Total sales/month', 'Organizational Unit', 'Replaceable', 'Note',
       'Email address', 'External (BE) Cont

In [13]:
df = df.drop(columns=[
    'DAS ID', 'FL/Subco', 'Subco Agency', 'FirstName', 'LastName', 'Employee Last name + First name', 'Gender',
    'StartDate', 'EndDate', 'Dummy\n/\nRoma Nr.', 'WBS',
    'Jan 25 FTE Count ', 'Feb 25 FTE Count ', 'Mar 25 FTE Count ',
    'Apr 25 FTE Count ', 'May 25 FTE Count ', 'Jun 25 FTE Count',
    'Jul 25 FTE Count ', 'Aug 25 FTE Count ', 'Sep 25 FTE Count ',
    'Oct 25 FTE Count ', 'Nov 25 FTE Count ', 'Dec 25 FTE Count ',
    'Customer n°', 'Customer', 'Contract Type', 'Hourly\nSales Rate', '% Margin',
    'Profile', 'Division', 'Location', 'Contractual Profile (10.A)',
    'Seniority', ' Sales ', 'Margin', 'Total cost/month',
    'Total sales/month', 'Organizational Unit', 'Replaceable', 'Note',
    'Email address', 'External (BE) Contract End Date', 'PM',
    'Clause Travel expense in SA', 'das profile', 'Overtime allowed',
    'Standby allowed'
])

In [14]:
df.head(0)


,NESSIE ID,Internal or external employee,Country,"Employee First name + Last name (without ""LEFT"")",Contract,GCM Level,Hourly PURCHASE PRICE,Competence,Cost,active_flag


3. Rename columns to match SQL bronze layer structure.

In [15]:
df.columns

Index(['NESSIE ID', 'Internal or external employee', 'Country',
       'Employee First name + Last name (without "LEFT")', 'Contract',
       'GCM Level', 'Hourly PURCHASE PRICE', 'Competence', ' Cost ',
       'active_flag'],
      dtype='object')

In [16]:
df = df.rename(columns={
    'NESSIE ID':'nessie', 
    'Internal or external employee':'employment_type', 
    'Country':'country',
    'Employee First name + Last name (without "LEFT")':'name', 
    'Contract':'contract',
    'GCM Level':'gcm_level', 
    'Hourly PURCHASE PRICE':'hourly_rate', 
    'Competence':'competence', 
    ' Cost ':'daily_rate',
})

In [17]:
df.head(0)

,nessie,employment_type,country,name,contract,gcm_level,hourly_rate,competence,daily_rate,active_flag


4. Check numeric columns, replace NaN values and cast the data type if necessary.

In [18]:
# ensure that column nessie is an object(!)
df['nessie'].dtype
# cast the data type from str to numeric, if necessary
df['nessie'] = pd.to_numeric(df['nessie'])
# count NaN values
df['nessie'].value_counts(dropna=False)
# replace NaN values with 0
df['nessie'] = df['nessie'].fillna(0)
# df['gcm_level'] = df['gcm_level'].fillna(0)

# format numeric columns
# copilot helped here 🫡
df['nessie'] = df['nessie'].map(lambda x:'{:.0f}'.format(x) if x.is_integer() else str(x))
df['gcm_level'] = df['gcm_level'].map(lambda x:'{:.0f}'.format(x) if x.is_integer() else str(x))

5. Extract competence and contract and drop unnecessary columns.

In [19]:
# extract competence and drop unnecessary columns
df['competence'] = df['competence'].str.split(',', expand=True).drop([1, 2, 3, 4, 5], axis=1)
# contract should contain 3 letters - extract it
df['contract'] = df['contract'].str.split(' ', expand=True).drop([1, 2], axis=1)

6. Set new index.

In [20]:
df.set_index('nessie')
df.head(0)

,nessie,employment_type,country,name,contract,gcm_level,hourly_rate,competence,daily_rate,active_flag


7. Remove trailing spaces from strings.

In [21]:
# remove trailing spaces
df['name'] = df['name'].str.strip()
df['competence'] = df['competence'].str.strip()

8. Remove EUR symbols and cast the data type from str to float.<br>
`\s` removes any whitespace, `regex=True` ensures the pattern works as a regex.

In [ ]:
df['hourly_rate'] = df['hourly_rate'].str.replace(r'[€\s]', '', regex=True).astype(float)
df['daily_rate'] = df['daily_rate'].str.replace(r'[€\s]', '', regex=True)

9. Save as CSV file.

In [ ]:
# save on desktop
df.to_csv('wfm.csv', sep=',', encoding='utf-8', index=False, header=True)